## Modelhubs

- [https://huggingface.co/models](https://huggingface.co/models)
- [https://pytorch.org/hub/](https://pytorch.org/hub/)
- [https://www.tensorflow.org/hub](https://www.tensorflow.org/hub)

## Sentiment-analysis

In [ ]:
# conda install transformers | pip install transformers
from transformers import pipeline # Pipeline will abstract away the details.

In [ ]:
# pip install tf_keras
import tf_keras as keras

In [ ]:
classifier = pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english", device="mps") # Apple M-series
#classifier = pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english", device=0) # Cuda

In [ ]:
result = classifier("Python is very useful for certain things")

In [ ]:
print(result)

## Text-generation

In [ ]:
generator = pipeline("text-generation", model="distilgpt2", device="mps")

In [ ]:
result = generator("Bergen is a very",
                   max_length=40,
                   num_return_sequences=2,
                   truncation=True
                  )

In [ ]:
print(result)

## Zero-shot-classification

In [ ]:
zsclassifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device="mps")

In [ ]:
result = zsclassifier(
    "AI is like a teacher that you want to trust, but you know it can fail",
    candidate_labels=["technology", "religion", "politics"]
)

In [ ]:
print(result)

## Tokenizer & model

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
classifier = pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english", device="mps") # Apple M-series
#classifier = pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english", device=0) # Cuda

In [ ]:
result = classifier("Python is very useful for certain things")

In [ ]:
print(result)

In [ ]:
model_name = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device="mps") # Apple M-series

result = classifier("Python is very useful for certain things")
print(result)

In [ ]:
sequence = "The transformers library is awesome"
result = tokenizer(sequence)
print(result)

tokens = tokenizer.tokenize(sequence)
print("Tokens: ", tokens)

ids = tokenizer.convert_tokens_to_ids(tokens)
print("Ids: ", ids)

decoded_string = tokenizer.decode(ids)
print("Decoded string: ", decoded_string)

## Pytorch

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
model_name = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

X_train = ["Python is very useful for certain things.",
        "AI is a smallsubset of what we call technology."]

result = classifier(X_train)
print(result)

In [ ]:
batch = tokenizer(X_train, padding=True, truncation=True, max_length=512, return_tensors="pt")
print(type(batch))
print(batch)

In [ ]:
with torch.no_grad():
    outputs = model(**batch)
    print(outputs)
    
    predictions = F.softmax(outputs.logits, dim=1)
    print(predictions)
    
    labels = torch.argmax(predictions, dim=1)
    print(labels)

## Save and load

In [ ]:
save_path = "resources"

tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)

loaded_tokenizer = AutoTokenizer.from_pretrained(save_path)
loaded_model = AutoModelForSequenceClassification.from_pretrained(save_path)


## More examples

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

text = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""
print(summarizer(text, max_length=130, min_length=30, do_sample=False))

In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

In [ ]:
image

In [ ]:
from diffusers import StableDiffusionPipeline # pip install diffusers | pip install accelerate
import torch

model_id = "sd-legacy/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("mps")

# Recommended if your computer has < 64 GB of RAM
#pipe.enable_attention_slicing()

prompt = "a photo of an astronaut riding a horse on mars"
image = pipe(prompt).images[0]  

image.save("astronaut_rides_horse.png")


In [ ]:
image